In [1]:
from PIL import Image,ImageEnhance
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv('ans.csv')

In [3]:
df1.dropna(axis=1,inplace=True)
df1

,filename,value
0,501.bmp,5842
1,502.bmp,2291
2,503.bmp,5774
3,504.bmp,3514
4,505.bmp,9236
...,...,...
997,496.bmp,6736
998,497.bmp,2996
999,498.bmp,3486
1000,499.bmp,7292


In [4]:
#IMG
y_train=[]
x_train=[]
for i in range(df1.shape[0]):
    img = Image.open('epoch/' + df1['filename'][i]).convert('L')
    enhancer = ImageEnhance.Contrast(img)
    contrast = 10
    img = enhancer.enhance(contrast)
    img = np.asarray(img)
    x_train.append(img)
    y_train.append(str(df1['value'][i]))

In [5]:
#Value
for i in range(df1.shape[0]):
    temp = np.zeros((4,10))
    
    for j in range(4):
        temp[j][int(y_train[i][j])] = 1
        
    y_train[i] = temp

y_train_final=[[],[],[],[]]
for i in range(df1.shape[0]):
    for j in range(4):
        y_train_final[j].append(y_train[i][j])
y_train = [i for i in np.asarray(y_train_final)]

In [6]:
x_train = np.array(x_train).reshape(1002,24,124,1)
#y_train = y_train_final
print('shape of x_train : ',x_train.shape)
#print('shape of y_train : ',y_train.size)

shape of x_train :  (1002, 24, 124, 1)


In [7]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = InteractiveSession(config=config)

In [8]:
inf = layers.Input((24,124,1))
out = inf
out = layers.Conv2D(32,3,padding='same',activation='relu')(out)
out = layers.Conv2D(32,3,padding='same',activation='relu')(out)
out = layers.BatchNormalization()(out)
out = layers.MaxPooling2D(pool_size=(2,2))(out)


out = layers.Conv2D(64,3,padding='same',activation='relu')(out)
out = layers.Conv2D(64,3,padding='same',activation='relu')(out)
out = layers.BatchNormalization()(out)
out = layers.MaxPooling2D(pool_size=(2,2))(out)

out = layers.Conv2D(64,3,padding='same',activation='relu')(out)
out = layers.Conv2D(64,3,padding='same',activation='relu')(out)
out = layers.BatchNormalization()(out)
out = layers.MaxPooling2D(pool_size=(2,2))(out)

out = layers.Conv2D(128,3,padding='same',activation='relu')(out)
out = layers.Conv2D(128,3,padding='same',activation='relu')(out)
out = layers.BatchNormalization()(out)
out = layers.MaxPooling2D(pool_size=(2,2))(out)

out=layers.Flatten()(out)
out=layers.Dropout(0.4)(out)
out=layers.Dense(1024,activation='relu')(out)
out=[layers.Dense(10,activation='softmax')(out),layers.Dense(10,activation='softmax')(out),layers.Dense(10,activation='softmax')(out),layers.Dense(10,activation='softmax')(out)]
cnn=models.Model(inf,out)
#cnn.summary()

In [9]:
cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn.fit(x_train,y_train,epochs=200,batch_size=32,verbose=1)
cnn.save('crack.h5')

Epoch 1/200
32/32 [==============================] - 0s 15ms/step - loss: 8.7524 - dense_1_loss: 2.0277 - dense_2_loss: 2.1758 - dense_3_loss: 2.2300 - dense_4_loss: 2.3188 - dense_1_accuracy: 0.3333 - dense_2_accuracy: 0.2754 - dense_3_accuracy: 0.2575 - dense_4_accuracy: 0.2475
Epoch 2/200
32/32 [==============================] - 0s 10ms/step - loss: 2.9884 - dense_1_loss: 0.6010 - dense_2_loss: 0.7746 - dense_3_loss: 0.7915 - dense_4_loss: 0.8213 - dense_1_accuracy: 0.8044 - dense_2_accuracy: 0.7226 - dense_3_accuracy: 0.7315 - dense_4_accuracy: 0.7126
Epoch 3/200
32/32 [==============================] - 0s 10ms/step - loss: 0.8590 - dense_1_loss: 0.1837 - dense_2_loss: 0.2022 - dense_3_loss: 0.2364 - dense_4_loss: 0.2368 - dense_1_accuracy: 0.9571 - dense_2_accuracy: 0.9401 - dense_3_accuracy: 0.9291 - dense_4_accuracy: 0.9321
Epoch 4/200
32/32 [==============================] - 0s 10ms/step - loss: 0.4094 - dense_1_loss: 0.1075 - dense_2_loss: 0.0656 - dense_3_loss: 0.1272 - dense

32/32 [==============================] - 0s 10ms/step - loss: 0.0357 - dense_1_loss: 0.0087 - dense_2_loss: 0.0059 - dense_3_loss: 0.0117 - dense_4_loss: 0.0094 - dense_1_accuracy: 0.9970 - dense_2_accuracy: 0.9990 - dense_3_accuracy: 0.9970 - dense_4_accuracy: 0.9970  
Epoch 31/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0274 - dense_1_loss: 0.0030 - dense_2_loss: 0.0039 - dense_3_loss: 0.0078 - dense_4_loss: 0.0127 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9970 - dense_4_accuracy: 0.9970  
Epoch 32/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0426 - dense_1_loss: 2.8056e-04 - dense_2_loss: 0.0127 - dense_3_loss: 0.0225 - dense_4_loss: 0.0071 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9950 - dense_3_accuracy: 0.9940 - dense_4_accuracy: 0.9970
Epoch 33/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0243 - dense_1_loss: 6.1456e-04 - dense_2_loss: 0.0097 - dense_3_loss: 0.0118 - den

32/32 [==============================] - 0s 10ms/step - loss: 0.0185 - dense_1_loss: 0.0102 - dense_2_loss: 0.0059 - dense_3_loss: 0.0016 - dense_4_loss: 8.8352e-04 - dense_1_accuracy: 0.9950 - dense_2_accuracy: 0.9970 - dense_3_accuracy: 0.9990 - dense_4_accuracy: 1.0000
Epoch 59/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0301 - dense_1_loss: 0.0051 - dense_2_loss: 0.0116 - dense_3_loss: 0.0013 - dense_4_loss: 0.0121 - dense_1_accuracy: 0.9980 - dense_2_accuracy: 0.9950 - dense_3_accuracy: 0.9990 - dense_4_accuracy: 0.9980          
Epoch 60/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0241 - dense_1_loss: 0.0153 - dense_2_loss: 0.0043 - dense_3_loss: 0.0016 - dense_4_loss: 0.0028 - dense_1_accuracy: 0.9960 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9990 - dense_4_accuracy: 0.9990          
Epoch 61/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0131 - dense_1_loss: 0.0066 - dense_2_loss: 8.5095e-04 - dense_3_l

32/32 [==============================] - 0s 10ms/step - loss: 0.0440 - dense_1_loss: 0.0112 - dense_2_loss: 0.0078 - dense_3_loss: 0.0153 - dense_4_loss: 0.0098 - dense_1_accuracy: 0.9970 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9960 - dense_4_accuracy: 0.9980   
Epoch 87/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0551 - dense_1_loss: 0.0040 - dense_2_loss: 0.0232 - dense_3_loss: 0.0051 - dense_4_loss: 0.0228 - dense_1_accuracy: 0.9990 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9980 - dense_4_accuracy: 0.9940          
Epoch 88/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0406 - dense_1_loss: 8.9489e-05 - dense_2_loss: 0.0196 - dense_3_loss: 4.1956e-04 - dense_4_loss: 0.0205 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9970 - dense_3_accuracy: 1.0000 - dense_4_accuracy: 0.9960  
Epoch 89/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0202 - dense_1_loss: 0.0082 - dense_2_loss: 0.0085 - dense_3_loss: 

32/32 [==============================] - 0s 10ms/step - loss: 0.0700 - dense_1_loss: 0.0149 - dense_2_loss: 0.0109 - dense_3_loss: 0.0226 - dense_4_loss: 0.0215 - dense_1_accuracy: 0.9970 - dense_2_accuracy: 0.9970 - dense_3_accuracy: 0.9980 - dense_4_accuracy: 0.9970
Epoch 115/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0688 - dense_1_loss: 0.0158 - dense_2_loss: 0.0175 - dense_3_loss: 0.0118 - dense_4_loss: 0.0238 - dense_1_accuracy: 0.9970 - dense_2_accuracy: 0.9950 - dense_3_accuracy: 0.9950 - dense_4_accuracy: 0.9960       
Epoch 116/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0375 - dense_1_loss: 2.3783e-04 - dense_2_loss: 0.0064 - dense_3_loss: 0.0278 - dense_4_loss: 0.0031 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9970 - dense_3_accuracy: 0.9980 - dense_4_accuracy: 0.9990
Epoch 117/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0434 - dense_1_loss: 0.0019 - dense_2_loss: 0.0062 - dense_3_loss: 0.0173 - de

32/32 [==============================] - 0s 9ms/step - loss: 0.0561 - dense_1_loss: 0.0055 - dense_2_loss: 0.0093 - dense_3_loss: 0.0095 - dense_4_loss: 0.0319 - dense_1_accuracy: 0.9990 - dense_2_accuracy: 0.9990 - dense_3_accuracy: 0.9970 - dense_4_accuracy: 0.9980   
Epoch 143/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0452 - dense_1_loss: 0.0174 - dense_2_loss: 5.9946e-05 - dense_3_loss: 0.0277 - dense_4_loss: 6.7107e-05 - dense_1_accuracy: 0.9960 - dense_2_accuracy: 1.0000 - dense_3_accuracy: 0.9960 - dense_4_accuracy: 1.0000    
Epoch 144/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0305 - dense_1_loss: 0.0193 - dense_2_loss: 0.0027 - dense_3_loss: 0.0027 - dense_4_loss: 0.0058 - dense_1_accuracy: 0.9970 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9990 - dense_4_accuracy: 0.9980           
Epoch 145/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0684 - dense_1_loss: 0.0145 - dense_2_loss: 0.0223 - dense_3_loss

32/32 [==============================] - 0s 10ms/step - loss: 0.0157 - dense_1_loss: 3.0377e-05 - dense_2_loss: 0.0052 - dense_3_loss: 6.4580e-04 - dense_4_loss: 0.0098 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9990 - dense_3_accuracy: 1.0000 - dense_4_accuracy: 0.9970  
Epoch 171/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0372 - dense_1_loss: 5.1327e-04 - dense_2_loss: 0.0255 - dense_3_loss: 8.1123e-04 - dense_4_loss: 0.0103 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 1.0000 - dense_4_accuracy: 0.9980   
Epoch 172/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0170 - dense_1_loss: 8.1323e-06 - dense_2_loss: 0.0062 - dense_3_loss: 6.2447e-04 - dense_4_loss: 0.0102 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 0.9990 - dense_3_accuracy: 1.0000 - dense_4_accuracy: 0.9980    
Epoch 173/200
32/32 [==============================] - 0s 10ms/step - loss: 0.0043 - dense_1_loss: 4.7345e-04 - dense_2_loss: 2.5554e

32/32 [==============================] - 0s 9ms/step - loss: 0.0607 - dense_1_loss: 0.0228 - dense_2_loss: 0.0109 - dense_3_loss: 0.0116 - dense_4_loss: 0.0154 - dense_1_accuracy: 0.9980 - dense_2_accuracy: 0.9980 - dense_3_accuracy: 0.9960 - dense_4_accuracy: 0.9960   
Epoch 199/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0633 - dense_1_loss: 0.0374 - dense_2_loss: 0.0170 - dense_3_loss: 5.8789e-04 - dense_4_loss: 0.0083 - dense_1_accuracy: 0.9960 - dense_2_accuracy: 0.9990 - dense_3_accuracy: 1.0000 - dense_4_accuracy: 0.9990       
Epoch 200/200
32/32 [==============================] - 0s 9ms/step - loss: 0.0206 - dense_1_loss: 7.0518e-04 - dense_2_loss: 6.6131e-05 - dense_3_loss: 0.0187 - dense_4_loss: 0.0011 - dense_1_accuracy: 1.0000 - dense_2_accuracy: 1.0000 - dense_3_accuracy: 0.9970 - dense_4_accuracy: 0.9990   
